In [15]:
import pandas as pd
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true") 
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/02 23:16:50 WARN Utils: Your hostname, DESKTOP-9VIST87 resolves to a loopback address: 127.0.1.1; using 172.28.233.244 instead (on interface eth0)
22/10/02 23:16:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/02 23:16:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/02 23:16:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/02 23:16:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
transaction_df = spark.read.parquet('../data/curated/transactions_detail.parquet')
transaction_df.limit(10)

user_id,dollar_value,order_id,merchant_name,merchant_abn,take_rate,type,consumer_address,state,postcode,gender
4,157.0,5b867ac9-d1e0-430...,Lectus Limited,24015173965,6.79,"cable, satellite,...",00653 Davenport C...,NSW,2780,Female
4,119.86580481272857,6137f115-68c8-425...,Ullamcorper Velit...,95508140753,3.88,"cable, satellite,...",00653 Davenport C...,NSW,2780,Female
4,49.76743020022073,161f3485-8ccf-46a...,Tempor Inc.,61412665910,1.75,"furniture, home f...",00653 Davenport C...,NSW,2780,Female
4,45.73101703759395,11a57360-c6e8-48a...,Euismod Urna Inco...,54874856789,6.28,"hobby, toy and ga...",00653 Davenport C...,NSW,2780,Female
4,5.937376498119946,fc565084-2577-452...,Suspendisse Dui C...,46804135891,2.93,"opticians, optica...",00653 Davenport C...,NSW,2780,Female
4,506.941369622103,6e773630-6034-4dd...,Quisque Libero La...,93532380797,4.99,"computers, comput...",00653 Davenport C...,NSW,2780,Female
4,289.3209135411692,f1318f5a-e558-401...,Arcu Sed Eu Incor...,35909341340,4.8,computer programm...,00653 Davenport C...,NSW,2780,Female
4,148.08936648425507,3c4b8bfe-0781-414...,Tellus Nunc Lectu...,68028409124,2.66,florists supplies...,00653 Davenport C...,NSW,2780,Female
4,130.9177443060089,1e7b1473-446d-406...,Ultricies Digniss...,60956456424,4.69,"gift, card, novel...",00653 Davenport C...,NSW,2780,Female
4,1.5555004881188321,69a55b9e-520d-44b...,Lacus Consulting,45629217853,6.98,"gift, card, novel...",00653 Davenport C...,NSW,2780,Female


In [4]:
transaction_count_df = transaction_df.groupBy('merchant_name', 'postcode').count()
transaction_count_df = transaction_count_df.withColumnRenamed("count","transaction_count")

In [4]:
transaction_count_df.limit(10)

merchant_name,postcode,transaction_count
Faucibus Morbi Fo...,2323,4
Tempor Arcu Assoc...,3055,1
Libero Et Limited,6846,11
At Pede Inc.,6846,4
Nulla Facilisis I...,6846,11
A Ultricies Insti...,4055,3
Augue Eu LLP,4055,4
Interdum Feugiat ...,5277,10
Turpis In Incorpo...,5277,4
Suspendisse Sagit...,3690,4


In [5]:
from pyspark.sql import Window
import pyspark.sql.functions as f

w = Window.partitionBy('merchant_name')
max_count_df = transaction_count_df.withColumn('max_count', f.max('transaction_count').over(w))\
                .where(f.col('transaction_count') == f.col('max_count'))\
                .drop('max_count')\
                .dropDuplicates(['merchant_name'])


In [6]:
max_count_df.show()

+--------------------+--------+-----------------+
|       merchant_name|postcode|transaction_count|
+--------------------+--------+-----------------+
|        A Associates|    4620|                2|
|         A Neque LLP|    2047|                1|
|       A Nunc In LLP|    1172|                4|
|A Ultricies Found...|    6074|                5|
|      Ac Corporation|    3862|                3|
|Ac Fermentum Vel LLP|    1181|                2|
|      Ac Feugiat LLC|    6415|                2|
|Ac Turpis Egestas PC|    4018|                7|
|Adipiscing Elit C...|    6470|               13|
| Adipiscing Enim LLC|    1819|               20|
|     Adipiscing Inc.|    6475|                1|
|Adipiscing Lacus ...|    2799|                2|
|Adipiscing Lobort...|    2901|               11|
|Aenean Gravida In...|    3053|                1|
|Aliquam Auctor Ve...|    2336|                5|
|Aliquam Gravida M...|    2624|               13|
|         Aliquam Ltd|    4375|                1|


In [7]:
merchant_detail = spark.read.parquet('../data/curated/merchant_detail.parquet')

In [8]:
merchant_detail.limit(10)

merchant_abn,merchant_name,type,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit
79827781481,Amet Risus Inc.,"furniture, home f...",6.82,8657277.1,4251,2037.0,32801.0
48534649627,Dignissim Maecena...,"opticians, optica...",6.64,8316735.67,58685,142.0,30680.0
32361057556,Orci In Consequat...,"gift, card, novel...",6.61,8339994.52,75853,110.0,30626.0
86578477987,Leo In Consulting,"watch, clock, and...",6.43,8443178.7,241336,35.0,30161.0
38700038932,Etiam Bibendum In...,tent and awning s...,6.31,8482176.66,6341,1338.0,29735.0
45629217853,Lacus Consulting,"gift, card, novel...",6.98,7436925.45,201830,37.0,28839.0
96680767841,Ornare Limited,motor vehicle sup...,5.91,8679874.17,27541,315.0,28499.0
21439773999,Mauris Non Institute,"cable, satellite,...",6.1,8337853.96,106724,78.0,28256.0
63123845164,Odio Phasellus In...,artist supply and...,6.59,7570160.92,10075,751.0,27715.0
64403598239,Lobortis Ultrices...,music shops - mus...,6.31,7842635.61,100394,78.0,27493.0


In [9]:
merchant_detail = merchant_detail.join(max_count_df.drop(col("transaction_count")),['merchant_name'],how='inner')

In [10]:
merchant_detail.limit(10)

merchant_name,merchant_abn,type,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit,postcode
A Associates,14955471775,"opticians, optica...",4.95,24797.58,71,349.0,68.0,4620
A Neque LLP,79081516258,antique shops - s...,3.45,58451.53,5,11690.0,112.0,2047
A Nunc In LLP,52069051820,bicycle shops - s...,2.17,52071.84,1045,50.0,63.0,1172
A Ultricies Found...,34984654037,digital goods: bo...,2.34,106373.83,1224,87.0,138.0,6074
Ac Corporation,87178039859,"gift, card, novel...",3.35,40765.81,721,57.0,76.0,3862
Ac Fermentum Vel LLP,17435313552,"stationery, offic...",2.48,34385.67,50,688.0,47.0,1181
Ac Feugiat LLC,30403523748,"computers, comput...",6.21,46009.53,153,301.0,159.0,6415
Ac Turpis Egestas PC,49758515423,"watch, clock, and...",6.26,393842.58,3390,116.0,1370.0,4018
Adipiscing Elit C...,49125619545,"cable, satellite,...",5.66,1021916.46,8636,118.0,3213.0,6470
Adipiscing Enim LLC,89502033586,digital goods: bo...,5.98,1224795.02,17351,71.0,4069.0,1819


In [11]:
merchant_detail.write.mode('overwrite').parquet('../data/curated/new_merchant_detail.parquet')

In [27]:
new_merchant_detail = pd.read_parquet('../data/curated/new_merchant_detail.parquet')
new_merchant_detail['postcode'] = new_merchant_detail['postcode'].astype(int)
new_merchant_detail

,merchant_name,merchant_abn,type,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit,postcode
0,A Associates,14955471775,"opticians, optical goods, and eyeglasses",4.95,24797.58,71,349.0,68.0,4620
1,A Neque LLP,79081516258,"antique shops - sales, repairs, and restoratio...",3.45,58451.53,5,11690.0,112.0,2047
2,A Nunc In LLP,52069051820,bicycle shops - sales and service,2.17,52071.84,1045,50.0,63.0,1172
3,A Ultricies Foundation,34984654037,"digital goods: books, movies, music",2.34,106373.83,1224,87.0,138.0,6074
4,Ac Corporation,87178039859,"gift, card, novelty, and souvenir shops",3.35,40765.81,721,57.0,76.0,3862
...,...,...,...,...,...,...,...,...,...
4021,Vel Lectus Cum LLC,61112604295,"equipment, tool, furniture, and appliance rent...",1.70,43266.06,1228,35.0,41.0,4053
4022,Vel Turpis Aliquam Company,52594457116,bicycle shops - sales and service,5.51,3504961.29,6864,511.0,10729.0,3822
4023,Vitae Orci Phasellus Foundation,55883454313,"opticians, optical goods, and eyeglasses",0.97,31292.98,415,75.0,17.0,837
4024,Vitae Orci Phasellus LLP,59614227972,"lawn and garden supply outlets, including nurs...",3.72,112488.42,93,1210.0,232.0,3128


In [26]:
income_df = pd.read_csv("../data/income/2021_income.csv")
income_df['POA_CODE_2021'] = income_df['POA_CODE_2021'].astype(int)
income_df

,POA_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
0,2000,32,2800,941,625,2367,1.3,2225,2.1
1,2007,30,2500,772,500,2197,1.2,1805,2.1
2,2008,28,2600,860,525,2453,1.2,1746,1.9
3,2009,37,2800,1297,580,3035,1.1,2422,2.1
4,2010,36,2900,1479,550,3709,1.1,2297,1.7
...,...,...,...,...,...,...,...,...,...
2638,2899,50,1300,736,240,1630,0.8,1184,2.1
2639,6798,38,1300,1233,150,2659,1.0,2109,2.5
2640,6799,40,1972,741,231,1678,1.1,2519,3.8
2641,9494,0,0,0,0,0,0.0,0,0.0


In [28]:
merged_df = new_merchant_detail \
    .merge(income_df[['POA_CODE_2021', 'Median_tot_prsnl_inc_weekly']], left_on='postcode', right_on='POA_CODE_2021') \
    .drop('POA_CODE_2021', axis=1)


In [31]:
merged_df

,merchant_name,merchant_abn,type,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit,postcode,Median_tot_prsnl_inc_weekly
0,A Associates,14955471775,"opticians, optical goods, and eyeglasses",4.95,24797.58,71,349.0,68.0,4620,476
1,At Auctor Ullamcorper Incorporated,21702179125,"gift, card, novelty, and souvenir shops",2.65,117130.21,1280,92.0,172.0,4620,476
2,Ante Vivamus Industries,57540672941,bicycle shops - sales and service,4.71,490063.47,609,805.0,1282.0,4620,476
3,A Felis Ullamcorper Corporation,36800720158,"florists supplies, nursery stock, and flowers",6.04,509614.30,1155,441.0,1710.0,4620,476
4,A Neque LLP,79081516258,"antique shops - sales, repairs, and restoratio...",3.45,58451.53,5,11690.0,112.0,2047,1388
...,...,...,...,...,...,...,...,...,...,...
3374,Tempus Scelerisque Lorem Incorporated,15061716544,"digital goods: books, movies, music",5.73,837855.82,15944,53.0,2667.0,2787,734
3375,Ultrices Industries,69431337574,telecom,1.64,34215.10,8,4277.0,31.0,3300,695
3376,Ut Molestie Incorporated,93095540883,"opticians, optical goods, and eyeglasses",4.39,10489.21,53,198.0,26.0,4671,470
3377,Vitae Orci Phasellus Foundation,55883454313,"opticians, optical goods, and eyeglasses",0.97,31292.98,415,75.0,17.0,837,736


In [34]:
merged_df.to_csv(r'../data/curated/merchant_detail.csv', index = False, header=True)